<a href="https://colab.research.google.com/github/tuomaseerola/onsetsync/blob/master/onsetsync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# onsetsync

A Jupyter notebook with code examples in R for _onsetsync_ package by [Tuomas Eerola](https://www.durham.ac.uk/staff/tuomas-eerola/), [Music and Science Lab]() at [Durham University](https://www.durham.ac.uk).

To run the code in your browser, open the file in Colab (click the icon "Open in Colab"). Alternatively, you can download the notebook and run it locally.

File `onsetsync.ipynb` | Version `7/11/2022` |

---

## Load libraries

In [ ]:
library(ggplot2,quietly = TRUE)
library(tidyverse,quietly = TRUE)
library(httr,quietly = TRUE)   # for reading data from online sources

devtools::install_github("tuomaseerola/onsetsync")
library(onsetsync,quietly = TRUE)
ver <- packageVersion('onsetsync')
print(ver)

## Load data
Dataset that comes with the package (Cuban Salsa and Son, Song 2 from IEMP corpus)

In [ ]:
CSS_Song2 <- onsetsync::CSS_IEMP[[2]]   # Read one song from internal data
CSS_Song2 <- dplyr::select(CSS_Song2,Label.SD,SD,Clave,Bass,Guitar,Tres,
                           CycleTime,Cycle,Isochronous.SD.Time) # Select some columns
print(head(CSS_Song2),format = "simple",digits = 3)

## Load data
Load the data directly from OSF.

In [ ]:
CSS_Song2_Onset <- get_OSF_csv('8a347') # Get onsets
print(head(CSS_Song2_Onset[,1:8,]))

One needs to add annotations and calculate the isochronous beats to the data

In [ ]:
CSS_Song2_Metre <- get_OSF_csv('4cdpr') # Annotations for cycles

CSS_Song2_Onset <- dplyr::select(CSS_Song2_Onset,
                                 Label.SD,SD,Clave,Bass,Guitar,Tres)

CSS_Song2 <- add_annotation(df = CSS_Song2_Onset,
                            annotation = CSS_Song2_Metre$Cycle,
                            time = CSS_Song2_Metre$Time,
                            reference = 'Label.SD')
# Add isochronous beats to the data frame
CSS_Song2 <- add_isobeats(df = CSS_Song2,
                          instr = 'CycleTime',
                          beat = 'SD')

colnames(CSS_Song2)[9] <- 'Ann.Iso' # Rename
# Add isochronous beats based on mean timing of guitar, tres, and clave

CSS_Song2 <- add_isobeats(df = CSS_Song2,
                          instr = c('Guitar','Tres','Clave'),
                          beat = 'SD')
print(head(CSS_Song2))

## Summarise data
Tell us how many onsets/instruments and what are the typical IOIs (Inter Onset Intervals) between the onsets (in ms).

In [ ]:
tab1 <- summarise_onsets(df = CSS_Song2,
                         instr = c('Clave','Bass','Guitar','Tres'))
print(tab1)

## Visualise onset structures
Show four selected instruments and display beats using SD (Sub-Divisions).

In [ ]:
fig1 <- plot_by_beat(df = CSS_Song2,
                     instr = c('Bass','Clave','Guitar','Tres'),
                     beat = 'SD',
                     virtual = 'Isochronous.SD.Time',
                     pcols = 2)
options(repr.plot.width=10, repr.plot.height=6)
print(fig1)

## Deviation from isochrony
As above, but add extra information that displays how much the mean onsets deviate (%) from isochronic beats.

In [ ]:
fig2 <- plot_by_beat(df = CSS_Song2,
                     instr = c('Bass','Tres'),
                     beat = 'SD',
                     virtual = 'Isochronous.SD.Time',
                     pcols=1,
                     griddeviations = TRUE)
options(repr.plot.width=10, repr.plot.height=6)
print(fig2)

## Synchrony between the instruments

In [ ]:
set.seed(1201) # set random seed
N <- 100 # Let's select 100 onsets
d1 <- sync_sample_paired(CSS_Song2,'Clave','Bass',N,1,'SD',TRUE)
print(paste('Mean asynchrony of',round(mean(d1$asynch*1000),1),
            'ms & standard deviation of',round(sd(d1$asynch*1000),1),'ms'))

## Visualise synchrony between the instruments

In [ ]:
inst <- c('Clave','Bass','Guitar','Tres') # Define instruments
dn <- sync_execute_pairs(CSS_Song2,inst,N,10,'SD')
print(plot_by_pair(dn))  # plot

## Descriptive Statistics of synchrony

In [ ]:
table3 <- data.frame(summarise_sync_by_pair(dn))
print(knitr::kable(table3,
                   digits = 1,
                   caption = 'Descriptives, t-test and p-value.'))

## Classic Measures of Synchrony
From classic literature (Rasch, 1979; 1988 Wing, 2014; Clayton et al., 2020)

In [ ]:
d <- sync_sample_paired(CSS_Song2,'Clave','Bass',N=0,beat = 'SD')
s <- summarise_sync(d)
S <- data.frame(s); St<-t(S); colnames(St) <- 'ms'
print(St)

# Periodicity

Estimate periodicity of guitar onsets

In [ ]:
extract <- dplyr::filter(CSS_Song2, Guitar >= 60 & Guitar < 65)
fig10 <- gaussify_onsets(extract$Guitar, wlen = 0.2, plot = TRUE)
options(repr.plot.width=10, repr.plot.height=3)
print(fig10$fig)

## Calculate Periodicity
Use autocorrelation, FFT, or **periodigram** to estimate the periodicity

In [ ]:
per <- periodicity(extract,instr='Guitar',freq_range=c(0.1,0.6))
options(repr.plot.width=4, repr.plot.height=4)
print(per$Figure)

## Summarise Periodicity
Convenience functions to extract peak of periodicity curve.


In [ ]:
psumm <- summarise_periodicity(per$Curve)
print(paste('Period is:', round(psumm$Per,3), 'seconds'))

print(paste('In BPM:',period_to_BPM(psumm$Per*2)))

## Synchrony with other variables

In [ ]:
CSS_Song2 <- CSS_Song2 %>%
  group_by(Cycle) %>%
  mutate(Duration = max(Isochronous.SD.Time) - min(Isochronous.SD.Time))
CSS_Song2 <- ungroup(CSS_Song2) # drop grouping structure

d1 <- sync_sample_paired(df = CSS_Song2,
                         INSTR1 = 'Clave',
                         INSTR2 = 'Bass',
                         N = 200,
                         BNum = 1,
                         beat = 'Duration')
fig6 <- plot_by_variable(d1,
                         meta = 'Clave-Bass Synchrony',
                         xlab = 'Cycle duration (in s)')
print(fig6)

## Synchrony across several performances

In [ ]:
corpus <- onsetsync::CSS_IEMP
D <- sync_sample_paired(corpus,'Tres','Bass',N=0,beat='SD')
D <- D$asynch
D$asynch_abs <- abs(D$asynch)*1000

In [ ]:
fig7 <- plot_by_dataset(D,'asynch_abs','name', box = TRUE)
print(fig7)

## Synchrony across time

In [ ]:
d2 <- sync_sample_paired(df = CSS_Song2,
                         INSTR1 = 'Tres',
                         INSTR2 = 'Bass',
                         N = 0,
                         beat = 'Cycle')
tmp <- data.frame(asynch=abs(d2$asynch)*1000,Cycle=d2$beatL)

fig8 <- plot_by_var_time(df = tmp,
                         var1 = 'Cycle',
                         var2 = 'asynch',
                         ylabel = 'Asynchrony')
fig8